In [10]:
import pandas as pd
import importlib
import utils 
from utils import *
from os import listdir
from os.path import isfile, join
from random import shuffle
# Reload the library
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\rayji\\OneDrive\\Documents\\GitHub\\ml-project-2-big-three\\utils.py'>

In [12]:
data_train = load_data()
test_data = load_test_data()

In [13]:
# Print the number of lines in each dataset
print(data_train.shape)
print(test_data.shape)

(200000, 2)
(10000, 1)


In [5]:
# Use read_pickle() to open the .pkl file
#vocab = pd.read_pickle('vocab.pkl')
#cooc_matrix = pd.read_pickle('cooc.pkl')

In [6]:
import scipy.sparse

# Assuming 'sparse_matrix' is your sparse matrix
# Replace row_index and col_index with the specific indices you want to access
row_index = 0
col_index = 0

# Access the element at the specified row and column indices
row = cooc_matrix.getrow(row_index)
element = row[0, col_index]

# 'element' now contains the value at the specified position in the sparse matrix
element

NameError: name 'cooc_matrix' is not defined

In [15]:
data_train

,text,label
0,<user> i dunno justin read my mention or not ....,1
1,"because your logic is so dumb , i won't even c...",1
2,""" <user> just put casper in a box ! "" looved t...",1
3,<user> <user> thanks sir > > don't trip lil ma...,1
4,visiting my brother tmr is the bestest birthda...,1
...,...,...
199995,can't wait to fake tan tonight ! hate being pale,0
199996,<user> darling i lost my internet connection ....,0
199997,kanguru defender basic 4 gb usb 2.0 flash driv...,0
199998,rizan is sad now,0


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_train['text'], data_train['label'], test_size=0.01, random_state=42)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform the text data into TF-IDF features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression(max_iter=5000)
model.fit(X_train_tfidf, y_train)

# Predict labels for the test set
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.808


In [17]:
y_test_pred = model.predict(vectorizer.transform(test_data['text']))

In [39]:
export_submission(y_test_pred)